In [1]:
import os
from time import time
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import re
import scipy
from mpi4py import MPI
#from qiskit_ibm_runtime.fake_provider import FakeManilaV2, FakeMelbourneV2, FakeMumbaiV2, FakeBrisbane, FakeTorino
#from qiskit_ibm_runtime.options import ZneOptions
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
#from qiskit_aer import AerSimulator
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity

import scipy



comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

def inite(elec,orb):
    config=[]
    list1=[]
    list1.append([e for e in range(elec)])
    #singles
    for x in range(elec):
        count=orb-elec
        while (count<orb):
            for e in range(elec):
                if x==e:
                    if x%2==0:
                        config.append(count)
                        count=count+2
                    else:
                        config.append(count+1)
                        count=count+2
                else:
                    config.append(e)

            list1.append(config)
            config=[]
    #doubles
    for x in range(elec):
        for y in range(x+1,elec):
            count1=orb-elec
            count2=orb-elec
            for count1 in range(elec, orb, 2):
                for count2 in range(elec, orb, 2):
                    cont=0
                    if count1==count2:
                        if (x%2)!=(y%2):
                            cont=1
                    else:
                        cont=1
                    if (x%2)==(y%2) and count2<count1:
                        cont=0
                    if cont==1:
                        for e in range(elec):
                            if x==e:
                                if x%2==0:
                                    config.append(count1)
                                else:
                                    config.append(count1+1)
                            elif y==e:
                                if y%2==0:
                                    config.append(count2)
                                else:
                                    config.append(count2+1)
                            else:
                                config.append(e)
                        list1.append(config)
                        config=[]
    return list1
r = 1.88973  # 1 Angstrom in Bohr
distances = np.linspace(0.6*r, 2.46*r, 19)  # from 1 to bohr

# Setup other constants
symbols  = ['N', 'H', 'H', 'H']
#gr_energies = []
excited_state_energies = []

    # Define the molecule
for d in distances:
    #geometry = np.array([[0.0, 0.0,  d],
                        #[0.0, 0.0, -0.5488*r]
                        #], requires_grad = False)
    #active_electrons = 6
    #active_orbitals = 6
    #charge = 0
    
    geometry = np.array([[0.0, 0.0,  0.0],
                        [d, 0.0, 0.0],
                        [-0.506*r, 0.876*r, 0.0],
                        [-0.506*r, -0.876*r, 0.0]
                        ], requires_grad = False)
    active_electrons = 4
    active_orbitals = 4
    charge = 0
  

    def gs_exact(symbols, coordinates, active_electrons, active_orbitals, charge, adapt_it):
        gr_energies = []
        ash_excitation = []
        H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
        
        hf_state = qml.qchem.hf_state(active_electrons, qubits)
        #Calculation of HF state
        dev = qml.device("lightning.qubit", wires=qubits)
        @qml.qnode(dev)
        def circuit(hf_state, active_electrons, qubits, H):
            #print('Updated hf_state is', hf_state)  
            qml.BasisState(hf_state, wires=range(qubits))
            return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
        
        # Commutator calculation for HF state
        @qml.qnode(dev)
        def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
            qml.BasisState(k, wires=range(qubits))
            res = qml.commutator(H, w)   #Calculating the commutator
            return qml.expval(res)
        
        # Commutator calculation for other states except HF state
        @qml.qnode(dev)
        def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
            qml.StatePrep(k, wires=range(qubits))
            res = qml.commutator(H, w) #Calculating the commutator
            return qml.expval(res)

        #Energy calculation 
        @qml.qnode(dev)
        def ash(params, ash_excitation, hf_state, H):
            [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
            for i, excitation in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
        
        # Calculation of New state, same as the above function but with the state return
        dev1 = qml.device("lightning.qubit", wires=qubits)
        @qml.qnode(dev1)
        def new_state(hf_state, ash_excitation, params):
            [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
            for i, excitations in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.state()
        
        def cost(params):
            energy = ash(params, ash_excitation, hf_state, H)
            return energy

        print('HF state is', circuit(hf_state, active_electrons, qubits, H))
        singles, doubles = qml.qchem.excitations(active_electrons, qubits)
        op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
        op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
        operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
        #print('Total excitations are', len(operator_pool))
        states = [hf_state]
        params = np.zeros(len(ash_excitation), requires_grad=True) 

        null_state = np.zeros(qubits,int)
        #print('Null state is', null_state)

        for j in range(adapt_it):
            #print('The adapt iteration now is', j)  #Adapt iteration
            max_value = float('-inf')
            max_operator = None
            k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
        
            for i in operator_pool:
                #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
                w = qml.fermi.jordan_wigner(i)  #JW transformation
                if np.array_equal(k, hf_state): # If the current state is the HF state
                    current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
                else:
                    current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

                if current_value > max_value:
                    max_value = current_value
                    max_operator = i

            indices_str = re.findall(r'\d+', str(max_operator))
            excitations = [int(index) for index in indices_str]
            #print('Highest gradient excitation is', excitations)
            ash_excitation.append(excitations) #Appending the excitations to the ash_excitation

            params = np.append(params, 0.0)  #Parameters initialization

            #Energy calculation
            result = minimize(cost, params, method='powell', tol = 1e-12, options = {'disp': False, 'maxiter': 1e8})

            params= (result.x)
            gr_energies.append(result.fun)


            ostate = new_state(hf_state, ash_excitation, params)
           
            states.append(ostate)
        print("gr_energies:", gr_energies[-1])
        return params, ash_excitation



    def qsc_eom(symbols, coordinates, active_electrons, active_orbitals, charge, params, ash_excitation, shots=0):
        # Build the electronic Hamiltonian
        H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method='pyscf', active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
        hf_state = qml.qchem.hf_state(active_electrons, qubits)
        
        wires=range(qubits)

        null_state = np.zeros(qubits,int)
        list1 = inite(active_electrons,qubits)
        if shots==0:
            dev = qml.device("lightning.qubit", wires=qubits)
           
        else:
            dev = qml.device("lightning.qubit", wires=qubits,shots=shots)
            
        #circuit for diagonal part
        @qml.qnode(dev)
        def circuit_d(params, occ,wires, hf_state, ash_excitation):
            for w in occ:
                qml.X(wires=w)
            for i, excitations in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.expval(H)
        
        #circuit for off-diagonal part
        @qml.qnode(dev)
        def circuit_od(params, occ1, occ2, wires, hf_state, ash_excitation):
            for w in occ1:
                qml.X(wires=w)
            first=-1
            for v in occ2:
                if v not in occ1:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            for v in occ1:
                if v not in occ2:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            for i, excitations in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.expval(H)

        #final M matrix
        mat_size = len(list1)
        
        block_size = mat_size/size
        lo_limit = int(rank * block_size)
        up_limit = int((rank+1) * block_size)
        if rank == size-1:
            up_limit = mat_size

        M = np.zeros((mat_size, mat_size))
        M_1d_size = int(mat_size * (mat_size + 1)/2)
        #eig = np.zeros(mat_size)
        Mdiag = np.zeros(mat_size)
        Mdiag_local = np.zeros(mat_size)
        M_local = np.zeros((mat_size, mat_size))
        for i in range(lo_limit, up_limit):
            Mdiag_local[i] = circuit_d(params, list1[i], wires, null_state, ash_excitation)
        comm.Reduce(Mdiag_local, Mdiag, MPI.SUM, 0)

        def idx(n):
            id_i = np.ceil((-1 + np.sqrt(1 + 4 * 2 * n))/2)
            id_j = n - (id_i-1) * (id_i)/2
            return int(id_i-1), int(id_j-1)

        for ii in range(rank, M_1d_size, size):
            c = ii + 1

            idi, idj = idx(c)

            if idi==idj:
                Mtmp = Mdiag[idi]
            else:
                #print(idi, idj)
                Mtmp = circuit_od(params, list1[idi], list1[idj], wires, null_state, ash_excitation) - Mdiag[idi]/2.0 - Mdiag[idj]/2.0

            M_local[idi, idj] = Mtmp
            M_local[idj, idi] = Mtmp

        comm.Reduce(M_local, M, MPI.SUM, 0)
        comm.Bcast(M, root=0)
    
        eig, evec=np.linalg.eig(M)
      
        return np.sort(eig), evec

    print(f"Computing for bond length: {d / r:.2f} Å")
    params, ash_excitation = gs_exact(symbols, geometry, active_electrons, active_orbitals, charge, adapt_it=10) #1 is used for params
    eig, evec = qsc_eom(symbols, geometry, active_electrons, active_orbitals, charge, params, ash_excitation)
#np.save('eigenvalues.npy', eig)
#np.save('eigenvectors.npy', evec)
    excited_state_energies.append(eig)
    print("qscEOM", excited_state_energies)

Computing for bond length: 0.60 Å
HF state is -55.50579854955699


/Users/prince.kwao/miniconda3/envs/prince/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:353: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


gr_energies: -55.519648902656144
qscEOM [tensor([-55.5196489 , -55.06435946, -54.98061064, -54.97108563,
        -54.94466433, -54.80078116, -54.73952572, -54.6690587 ,
        -54.50165249, -54.42403042, -54.35778224, -54.32147702,
        -54.2954278 , -54.26231175, -54.20959326, -54.1019908 ,
        -54.07989076, -54.05727894, -54.0554297 , -53.96728727,
        -53.95203865, -53.94187983, -53.92391799, -53.89092078,
        -53.87617127, -53.64536137, -53.58645312], requires_grad=True)]
Computing for bond length: 0.70 Å
HF state is -55.7762385068465
gr_energies: -55.79038457104941
qscEOM [tensor([-55.5196489 , -55.06435946, -54.98061064, -54.97108563,
        -54.94466433, -54.80078116, -54.73952572, -54.6690587 ,
        -54.50165249, -54.42403042, -54.35778224, -54.32147702,
        -54.2954278 , -54.26231175, -54.20959326, -54.1019908 ,
        -54.07989076, -54.05727894, -54.0554297 , -53.96728727,
        -53.95203865, -53.94187983, -53.92391799, -53.89092078,
        -53.876

In [2]:
import os
from time import time
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import re
import scipy
from mpi4py import MPI
#from qiskit_ibm_runtime.fake_provider import FakeManilaV2, FakeMelbourneV2, FakeMumbaiV2, FakeBrisbane, FakeTorino
#from qiskit_ibm_runtime.options import ZneOptions
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
#from qiskit_aer import AerSimulator
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity

import scipy



comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

def inite(elec,orb):
    config=[]
    list1=[]
    list1.append([e for e in range(elec)])
    #singles
    for x in range(elec):
        count=orb-elec
        while (count<orb):
            for e in range(elec):
                if x==e:
                    if x%2==0:
                        config.append(count)
                        count=count+2
                    else:
                        config.append(count+1)
                        count=count+2
                else:
                    config.append(e)

            list1.append(config)
            config=[]
    #doubles
    for x in range(elec):
        for y in range(x+1,elec):
            count1=orb-elec
            count2=orb-elec
            for count1 in range(elec, orb, 2):
                for count2 in range(elec, orb, 2):
                    cont=0
                    if count1==count2:
                        if (x%2)!=(y%2):
                            cont=1
                    else:
                        cont=1
                    if (x%2)==(y%2) and count2<count1:
                        cont=0
                    if cont==1:
                        for e in range(elec):
                            if x==e:
                                if x%2==0:
                                    config.append(count1)
                                else:
                                    config.append(count1+1)
                            elif y==e:
                                if y%2==0:
                                    config.append(count2)
                                else:
                                    config.append(count2+1)
                            else:
                                config.append(e)
                        list1.append(config)
                        config=[]
    return list1
r = 1.88973  # 1 Angstrom in Bohr
distances = np.linspace(0.6*r, 2.46*r, 19)  # from 1 to bohr

# Setup other constants
symbols  = ['N', 'H', 'H', 'H']
#gr_energies = []
excited_state_energies = []

    # Define the molecule
for d in distances:
    #geometry = np.array([[0.0, 0.0,  d],
                        #[0.0, 0.0, -0.5488*r]
                        #], requires_grad = False)
    #active_electrons = 6
    #active_orbitals = 6
    #charge = 0
    
    geometry = np.array([[0.0, 0.0,  0.0],
                        [d, 0.0, 0.0],
                        [-0.506*r, 0.876*r, 0.0],
                        [-0.506*r, -0.876*r, 0.0]
                        ], requires_grad = False)
    active_electrons = 4
    active_orbitals = 4
    charge = 0
  

    def gs_exact(symbols, coordinates, active_electrons, active_orbitals, charge, adapt_it):
        gr_energies = []
        ash_excitation = []
        H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
        
        hf_state = qml.qchem.hf_state(active_electrons, qubits)
        #Calculation of HF state
        dev = qml.device("lightning.qubit", wires=qubits)
        @qml.qnode(dev)
        def circuit(hf_state, active_electrons, qubits, H):
            #print('Updated hf_state is', hf_state)  
            qml.BasisState(hf_state, wires=range(qubits))
            return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
        
        # Commutator calculation for HF state
        @qml.qnode(dev)
        def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
            qml.BasisState(k, wires=range(qubits))
            res = qml.commutator(H, w)   #Calculating the commutator
            return qml.expval(res)
        
        # Commutator calculation for other states except HF state
        @qml.qnode(dev)
        def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
            qml.StatePrep(k, wires=range(qubits))
            res = qml.commutator(H, w) #Calculating the commutator
            return qml.expval(res)

        #Energy calculation 
        @qml.qnode(dev)
        def ash(params, ash_excitation, hf_state, H):
            [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
            for i, excitation in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
        
        # Calculation of New state, same as the above function but with the state return
        dev1 = qml.device("lightning.qubit", wires=qubits)
        @qml.qnode(dev1)
        def new_state(hf_state, ash_excitation, params):
            [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
            for i, excitations in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.state()
        
        def cost(params):
            energy = ash(params, ash_excitation, hf_state, H)
            return energy

        print('HF state is', circuit(hf_state, active_electrons, qubits, H))
        singles, doubles = qml.qchem.excitations(active_electrons, qubits)
        op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
        op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
        operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
        #print('Total excitations are', len(operator_pool))
        states = [hf_state]
        params = np.zeros(len(ash_excitation), requires_grad=True) 

        null_state = np.zeros(qubits,int)
        #print('Null state is', null_state)

        for j in range(adapt_it):
            #print('The adapt iteration now is', j)  #Adapt iteration
            max_value = float('-inf')
            max_operator = None
            k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
        
            for i in operator_pool:
                #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
                w = qml.fermi.jordan_wigner(i)  #JW transformation
                if np.array_equal(k, hf_state): # If the current state is the HF state
                    current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
                else:
                    current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

                if current_value > max_value:
                    max_value = current_value
                    max_operator = i

            indices_str = re.findall(r'\d+', str(max_operator))
            excitations = [int(index) for index in indices_str]
            #print('Highest gradient excitation is', excitations)
            ash_excitation.append(excitations) #Appending the excitations to the ash_excitation

            params = np.append(params, 0.0)  #Parameters initialization

            #Energy calculation
            result = minimize(cost, params, method='powell', tol = 1e-12, options = {'disp': False, 'maxiter': 1e8})

            params= (result.x)
            gr_energies.append(result.fun)


            ostate = new_state(hf_state, ash_excitation, params)
           
            states.append(ostate)
        print("gr_energies:", gr_energies[-1])
        return params, ash_excitation



    def qsc_eom(symbols, coordinates, active_electrons, active_orbitals, charge, params, ash_excitation, shots=0):
        # Build the electronic Hamiltonian
        H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method='pyscf', active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
        hf_state = qml.qchem.hf_state(active_electrons, qubits)
        
        wires=range(qubits)

        null_state = np.zeros(qubits,int)
        list1 = inite(active_electrons,qubits)
        if shots==0:
            dev = qml.device("lightning.qubit", wires=qubits)
           
        else:
            dev = qml.device("lightning.qubit", wires=qubits,shots=shots)
            
        #circuit for diagonal part
        @qml.qnode(dev)
        def circuit_d(params, occ,wires, hf_state, ash_excitation):
            for w in occ:
                qml.X(wires=w)
            for i, excitations in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.expval(H)
        
        #circuit for off-diagonal part
        @qml.qnode(dev)
        def circuit_od(params, occ1, occ2, wires, hf_state, ash_excitation):
            for w in occ1:
                qml.X(wires=w)
            first=-1
            for v in occ2:
                if v not in occ1:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            for v in occ1:
                if v not in occ2:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            for i, excitations in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.expval(H)

        #final M matrix
        mat_size = len(list1)
        
        block_size = mat_size/size
        lo_limit = int(rank * block_size)
        up_limit = int((rank+1) * block_size)
        if rank == size-1:
            up_limit = mat_size

        M = np.zeros((mat_size, mat_size))
        M_1d_size = int(mat_size * (mat_size + 1)/2)
        #eig = np.zeros(mat_size)
        Mdiag = np.zeros(mat_size)
        Mdiag_local = np.zeros(mat_size)
        M_local = np.zeros((mat_size, mat_size))
        for i in range(lo_limit, up_limit):
            Mdiag_local[i] = circuit_d(params, list1[i], wires, null_state, ash_excitation)
        comm.Reduce(Mdiag_local, Mdiag, MPI.SUM, 0)

        def idx(n):
            id_i = np.ceil((-1 + np.sqrt(1 + 4 * 2 * n))/2)
            id_j = n - (id_i-1) * (id_i)/2
            return int(id_i-1), int(id_j-1)

        for ii in range(rank, M_1d_size, size):
            c = ii + 1

            idi, idj = idx(c)

            if idi==idj:
                Mtmp = Mdiag[idi]
            else:
                #print(idi, idj)
                Mtmp = circuit_od(params, list1[idi], list1[idj], wires, null_state, ash_excitation) - Mdiag[idi]/2.0 - Mdiag[idj]/2.0

            M_local[idi, idj] = Mtmp
            M_local[idj, idi] = Mtmp

        comm.Reduce(M_local, M, MPI.SUM, 0)
        comm.Bcast(M, root=0)
    
        eig, evec=np.linalg.eig(M)
      
        return np.sort(eig), evec

    print(f"Computing for bond length: {d / r:.2f} Å")
    params, ash_excitation = gs_exact(symbols, geometry, active_electrons, active_orbitals, charge, adapt_it=10) #1 is used for params
    eig, evec = qsc_eom(symbols, geometry, active_electrons, active_orbitals, charge, params, ash_excitation, shots = 1000)
#np.save('eigenvalues.npy', eig)
#np.save('eigenvectors.npy', evec)
    excited_state_energies.append(eig)
    print("qscEOM1k", excited_state_energies)

Computing for bond length: 0.60 Å
HF state is -55.50579854955699


/Users/prince.kwao/miniconda3/envs/prince/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:353: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


gr_energies: -55.519648902656144
qscEOM1k [tensor([-55.52822469, -55.07011909, -54.99369874, -54.97770744,
        -54.95017758, -54.79313963, -54.75666274, -54.68240274,
        -54.5027452 , -54.40817895, -54.37590318, -54.34242229,
        -54.30687657, -54.25191759, -54.21695329, -54.14366562,
        -54.0677413 , -54.05748856, -54.02701089, -53.98063747,
        -53.95371139, -53.92955513, -53.91871929, -53.87123361,
        -53.84659408, -53.60551815, -53.56685045], requires_grad=True)]
Computing for bond length: 0.70 Å
HF state is -55.7762385068465
gr_energies: -55.79038457104941
qscEOM1k [tensor([-55.52822469, -55.07011909, -54.99369874, -54.97770744,
        -54.95017758, -54.79313963, -54.75666274, -54.68240274,
        -54.5027452 , -54.40817895, -54.37590318, -54.34242229,
        -54.30687657, -54.25191759, -54.21695329, -54.14366562,
        -54.0677413 , -54.05748856, -54.02701089, -53.98063747,
        -53.95371139, -53.92955513, -53.91871929, -53.87123361,
        -53

In [3]:
import os
from time import time
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import re
import scipy
from mpi4py import MPI
#from qiskit_ibm_runtime.fake_provider import FakeManilaV2, FakeMelbourneV2, FakeMumbaiV2, FakeBrisbane, FakeTorino
#from qiskit_ibm_runtime.options import ZneOptions
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
#from qiskit_aer import AerSimulator
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity

import scipy



comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

def inite(elec,orb):
    config=[]
    list1=[]
    list1.append([e for e in range(elec)])
    #singles
    for x in range(elec):
        count=orb-elec
        while (count<orb):
            for e in range(elec):
                if x==e:
                    if x%2==0:
                        config.append(count)
                        count=count+2
                    else:
                        config.append(count+1)
                        count=count+2
                else:
                    config.append(e)

            list1.append(config)
            config=[]
    #doubles
    for x in range(elec):
        for y in range(x+1,elec):
            count1=orb-elec
            count2=orb-elec
            for count1 in range(elec, orb, 2):
                for count2 in range(elec, orb, 2):
                    cont=0
                    if count1==count2:
                        if (x%2)!=(y%2):
                            cont=1
                    else:
                        cont=1
                    if (x%2)==(y%2) and count2<count1:
                        cont=0
                    if cont==1:
                        for e in range(elec):
                            if x==e:
                                if x%2==0:
                                    config.append(count1)
                                else:
                                    config.append(count1+1)
                            elif y==e:
                                if y%2==0:
                                    config.append(count2)
                                else:
                                    config.append(count2+1)
                            else:
                                config.append(e)
                        list1.append(config)
                        config=[]
    return list1
r = 1.88973  # 1 Angstrom in Bohr
distances = np.linspace(0.6*r, 2.46*r, 19)  # from 1 to bohr

# Setup other constants
symbols  = ['N', 'H', 'H', 'H']
#gr_energies = []
excited_state_energies = []

    # Define the molecule
for d in distances:
    #geometry = np.array([[0.0, 0.0,  d],
                        #[0.0, 0.0, -0.5488*r]
                        #], requires_grad = False)
    #active_electrons = 6
    #active_orbitals = 6
    #charge = 0
    
    geometry = np.array([[0.0, 0.0,  0.0],
                        [d, 0.0, 0.0],
                        [-0.506*r, 0.876*r, 0.0],
                        [-0.506*r, -0.876*r, 0.0]
                        ], requires_grad = False)
    active_electrons = 4
    active_orbitals = 4
    charge = 0
  

    def gs_exact(symbols, coordinates, active_electrons, active_orbitals, charge, adapt_it):
        gr_energies = []
        ash_excitation = []
        H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
        
        hf_state = qml.qchem.hf_state(active_electrons, qubits)
        #Calculation of HF state
        dev = qml.device("lightning.qubit", wires=qubits)
        @qml.qnode(dev)
        def circuit(hf_state, active_electrons, qubits, H):
            #print('Updated hf_state is', hf_state)  
            qml.BasisState(hf_state, wires=range(qubits))
            return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
        
        # Commutator calculation for HF state
        @qml.qnode(dev)
        def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
            qml.BasisState(k, wires=range(qubits))
            res = qml.commutator(H, w)   #Calculating the commutator
            return qml.expval(res)
        
        # Commutator calculation for other states except HF state
        @qml.qnode(dev)
        def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
            qml.StatePrep(k, wires=range(qubits))
            res = qml.commutator(H, w) #Calculating the commutator
            return qml.expval(res)

        #Energy calculation 
        @qml.qnode(dev)
        def ash(params, ash_excitation, hf_state, H):
            [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
            for i, excitation in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
        
        # Calculation of New state, same as the above function but with the state return
        dev1 = qml.device("lightning.qubit", wires=qubits)
        @qml.qnode(dev1)
        def new_state(hf_state, ash_excitation, params):
            [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
            for i, excitations in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.state()
        
        def cost(params):
            energy = ash(params, ash_excitation, hf_state, H)
            return energy

        print('HF state is', circuit(hf_state, active_electrons, qubits, H))
        singles, doubles = qml.qchem.excitations(active_electrons, qubits)
        op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
        op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
        operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
        #print('Total excitations are', len(operator_pool))
        states = [hf_state]
        params = np.zeros(len(ash_excitation), requires_grad=True) 

        null_state = np.zeros(qubits,int)
        #print('Null state is', null_state)

        for j in range(adapt_it):
            #print('The adapt iteration now is', j)  #Adapt iteration
            max_value = float('-inf')
            max_operator = None
            k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
        
            for i in operator_pool:
                #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
                w = qml.fermi.jordan_wigner(i)  #JW transformation
                if np.array_equal(k, hf_state): # If the current state is the HF state
                    current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
                else:
                    current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

                if current_value > max_value:
                    max_value = current_value
                    max_operator = i

            indices_str = re.findall(r'\d+', str(max_operator))
            excitations = [int(index) for index in indices_str]
            #print('Highest gradient excitation is', excitations)
            ash_excitation.append(excitations) #Appending the excitations to the ash_excitation

            params = np.append(params, 0.0)  #Parameters initialization

            #Energy calculation
            result = minimize(cost, params, method='powell', tol = 1e-12, options = {'disp': False, 'maxiter': 1e8})

            params= (result.x)
            gr_energies.append(result.fun)


            ostate = new_state(hf_state, ash_excitation, params)
           
            states.append(ostate)
        print("gr_energies:", gr_energies[-1])
        return params, ash_excitation



    def qsc_eom(symbols, coordinates, active_electrons, active_orbitals, charge, params, ash_excitation, shots=0):
        # Build the electronic Hamiltonian
        H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method='pyscf', active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
        hf_state = qml.qchem.hf_state(active_electrons, qubits)
        
        wires=range(qubits)

        null_state = np.zeros(qubits,int)
        list1 = inite(active_electrons,qubits)
        if shots==0:
            dev = qml.device("lightning.qubit", wires=qubits)
           
        else:
            dev = qml.device("lightning.qubit", wires=qubits,shots=shots)
            
        #circuit for diagonal part
        @qml.qnode(dev)
        def circuit_d(params, occ,wires, hf_state, ash_excitation):
            for w in occ:
                qml.X(wires=w)
            for i, excitations in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.expval(H)
        
        #circuit for off-diagonal part
        @qml.qnode(dev)
        def circuit_od(params, occ1, occ2, wires, hf_state, ash_excitation):
            for w in occ1:
                qml.X(wires=w)
            first=-1
            for v in occ2:
                if v not in occ1:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            for v in occ1:
                if v not in occ2:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            for i, excitations in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.expval(H)

        #final M matrix
        mat_size = len(list1)
        
        block_size = mat_size/size
        lo_limit = int(rank * block_size)
        up_limit = int((rank+1) * block_size)
        if rank == size-1:
            up_limit = mat_size

        M = np.zeros((mat_size, mat_size))
        M_1d_size = int(mat_size * (mat_size + 1)/2)
        #eig = np.zeros(mat_size)
        Mdiag = np.zeros(mat_size)
        Mdiag_local = np.zeros(mat_size)
        M_local = np.zeros((mat_size, mat_size))
        for i in range(lo_limit, up_limit):
            Mdiag_local[i] = circuit_d(params, list1[i], wires, null_state, ash_excitation)
        comm.Reduce(Mdiag_local, Mdiag, MPI.SUM, 0)

        def idx(n):
            id_i = np.ceil((-1 + np.sqrt(1 + 4 * 2 * n))/2)
            id_j = n - (id_i-1) * (id_i)/2
            return int(id_i-1), int(id_j-1)

        for ii in range(rank, M_1d_size, size):
            c = ii + 1

            idi, idj = idx(c)

            if idi==idj:
                Mtmp = Mdiag[idi]
            else:
                #print(idi, idj)
                Mtmp = circuit_od(params, list1[idi], list1[idj], wires, null_state, ash_excitation) - Mdiag[idi]/2.0 - Mdiag[idj]/2.0

            M_local[idi, idj] = Mtmp
            M_local[idj, idi] = Mtmp

        comm.Reduce(M_local, M, MPI.SUM, 0)
        comm.Bcast(M, root=0)
    
        eig, evec=np.linalg.eig(M)
      
        return np.sort(eig), evec

    print(f"Computing for bond length: {d / r:.2f} Å")
    params, ash_excitation = gs_exact(symbols, geometry, active_electrons, active_orbitals, charge, adapt_it=10) #1 is used for params
    eig, evec = qsc_eom(symbols, geometry, active_electrons, active_orbitals, charge, params, ash_excitation, shots = 10000)
#np.save('eigenvalues.npy', eig)
#np.save('eigenvectors.npy', evec)
    excited_state_energies.append(eig)
    print("qscEOM10k", excited_state_energies)

Computing for bond length: 0.60 Å
HF state is -55.50579854955699


/Users/prince.kwao/miniconda3/envs/prince/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:353: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


gr_energies: -55.519648902656144
qscEOM10k [tensor([-55.52049966, -55.06356614, -54.97956047, -54.96784911,
        -54.94718721, -54.8039211 , -54.74273851, -54.66644235,
        -54.50486827, -54.42577113, -54.35302049, -54.31713491,
        -54.28211034, -54.25542527, -54.20719275, -54.10688805,
        -54.07986464, -54.06692547, -54.05538085, -53.96450083,
        -53.9584659 , -53.93270701, -53.92270036, -53.8906132 ,
        -53.88188209, -53.64566245, -53.58283521], requires_grad=True)]
Computing for bond length: 0.70 Å
HF state is -55.7762385068465
gr_energies: -55.79038457104941
qscEOM10k [tensor([-55.52049966, -55.06356614, -54.97956047, -54.96784911,
        -54.94718721, -54.8039211 , -54.74273851, -54.66644235,
        -54.50486827, -54.42577113, -54.35302049, -54.31713491,
        -54.28211034, -54.25542527, -54.20719275, -54.10688805,
        -54.07986464, -54.06692547, -54.05538085, -53.96450083,
        -53.9584659 , -53.93270701, -53.92270036, -53.8906132 ,
        -

In [4]:
import os
from time import time
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import re
import scipy
from mpi4py import MPI
#from qiskit_ibm_runtime.fake_provider import FakeManilaV2, FakeMelbourneV2, FakeMumbaiV2, FakeBrisbane, FakeTorino
#from qiskit_ibm_runtime.options import ZneOptions
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
#from qiskit_aer import AerSimulator
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity

import scipy



comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

def inite(elec,orb):
    config=[]
    list1=[]
    list1.append([e for e in range(elec)])
    #singles
    for x in range(elec):
        count=orb-elec
        while (count<orb):
            for e in range(elec):
                if x==e:
                    if x%2==0:
                        config.append(count)
                        count=count+2
                    else:
                        config.append(count+1)
                        count=count+2
                else:
                    config.append(e)

            list1.append(config)
            config=[]
    #doubles
    for x in range(elec):
        for y in range(x+1,elec):
            count1=orb-elec
            count2=orb-elec
            for count1 in range(elec, orb, 2):
                for count2 in range(elec, orb, 2):
                    cont=0
                    if count1==count2:
                        if (x%2)!=(y%2):
                            cont=1
                    else:
                        cont=1
                    if (x%2)==(y%2) and count2<count1:
                        cont=0
                    if cont==1:
                        for e in range(elec):
                            if x==e:
                                if x%2==0:
                                    config.append(count1)
                                else:
                                    config.append(count1+1)
                            elif y==e:
                                if y%2==0:
                                    config.append(count2)
                                else:
                                    config.append(count2+1)
                            else:
                                config.append(e)
                        list1.append(config)
                        config=[]
    return list1
r = 1.88973  # 1 Angstrom in Bohr
distances = np.linspace(0.6*r, 2.46*r, 19)  # from 1 to bohr

# Setup other constants
symbols  = ['N', 'H', 'H', 'H']
#gr_energies = []
excited_state_energies = []

    # Define the molecule
for d in distances:
    #geometry = np.array([[0.0, 0.0,  d],
                        #[0.0, 0.0, -0.5488*r]
                        #], requires_grad = False)
    #active_electrons = 6
    #active_orbitals = 6
    #charge = 0
    
    geometry = np.array([[0.0, 0.0,  0.0],
                        [d, 0.0, 0.0],
                        [-0.506*r, 0.876*r, 0.0],
                        [-0.506*r, -0.876*r, 0.0]
                        ], requires_grad = False)
    active_electrons = 4
    active_orbitals = 4
    charge = 0
  

    def gs_exact(symbols, coordinates, active_electrons, active_orbitals, charge, adapt_it):
        gr_energies = []
        ash_excitation = []
        H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
        
        hf_state = qml.qchem.hf_state(active_electrons, qubits)
        #Calculation of HF state
        dev = qml.device("lightning.qubit", wires=qubits)
        @qml.qnode(dev)
        def circuit(hf_state, active_electrons, qubits, H):
            #print('Updated hf_state is', hf_state)  
            qml.BasisState(hf_state, wires=range(qubits))
            return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
        
        # Commutator calculation for HF state
        @qml.qnode(dev)
        def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
            qml.BasisState(k, wires=range(qubits))
            res = qml.commutator(H, w)   #Calculating the commutator
            return qml.expval(res)
        
        # Commutator calculation for other states except HF state
        @qml.qnode(dev)
        def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
            qml.StatePrep(k, wires=range(qubits))
            res = qml.commutator(H, w) #Calculating the commutator
            return qml.expval(res)

        #Energy calculation 
        @qml.qnode(dev)
        def ash(params, ash_excitation, hf_state, H):
            [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
            for i, excitation in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
        
        # Calculation of New state, same as the above function but with the state return
        dev1 = qml.device("lightning.qubit", wires=qubits)
        @qml.qnode(dev1)
        def new_state(hf_state, ash_excitation, params):
            [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
            for i, excitations in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.state()
        
        def cost(params):
            energy = ash(params, ash_excitation, hf_state, H)
            return energy

        print('HF state is', circuit(hf_state, active_electrons, qubits, H))
        singles, doubles = qml.qchem.excitations(active_electrons, qubits)
        op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
        op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
        operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
        #print('Total excitations are', len(operator_pool))
        states = [hf_state]
        params = np.zeros(len(ash_excitation), requires_grad=True) 

        null_state = np.zeros(qubits,int)
        #print('Null state is', null_state)

        for j in range(adapt_it):
            #print('The adapt iteration now is', j)  #Adapt iteration
            max_value = float('-inf')
            max_operator = None
            k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
        
            for i in operator_pool:
                #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
                w = qml.fermi.jordan_wigner(i)  #JW transformation
                if np.array_equal(k, hf_state): # If the current state is the HF state
                    current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
                else:
                    current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

                if current_value > max_value:
                    max_value = current_value
                    max_operator = i

            indices_str = re.findall(r'\d+', str(max_operator))
            excitations = [int(index) for index in indices_str]
            #print('Highest gradient excitation is', excitations)
            ash_excitation.append(excitations) #Appending the excitations to the ash_excitation

            params = np.append(params, 0.0)  #Parameters initialization

            #Energy calculation
            result = minimize(cost, params, method='powell', tol = 1e-12, options = {'disp': False, 'maxiter': 1e8})

            params= (result.x)
            gr_energies.append(result.fun)


            ostate = new_state(hf_state, ash_excitation, params)
           
            states.append(ostate)
        print("gr_energies:", gr_energies[-1])
        return params, ash_excitation



    def qsc_eom(symbols, coordinates, active_electrons, active_orbitals, charge, params, ash_excitation, shots=0):
        # Build the electronic Hamiltonian
        H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method='pyscf', active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
        hf_state = qml.qchem.hf_state(active_electrons, qubits)
        
        wires=range(qubits)

        null_state = np.zeros(qubits,int)
        list1 = inite(active_electrons,qubits)
        if shots==0:
            dev = qml.device("lightning.qubit", wires=qubits)
           
        else:
            dev = qml.device("lightning.qubit", wires=qubits,shots=shots)
            
        #circuit for diagonal part
        @qml.qnode(dev)
        def circuit_d(params, occ,wires, hf_state, ash_excitation):
            for w in occ:
                qml.X(wires=w)
            for i, excitations in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.expval(H)
        
        #circuit for off-diagonal part
        @qml.qnode(dev)
        def circuit_od(params, occ1, occ2, wires, hf_state, ash_excitation):
            for w in occ1:
                qml.X(wires=w)
            first=-1
            for v in occ2:
                if v not in occ1:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            for v in occ1:
                if v not in occ2:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            for i, excitations in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.expval(H)

        #final M matrix
        mat_size = len(list1)
        
        block_size = mat_size/size
        lo_limit = int(rank * block_size)
        up_limit = int((rank+1) * block_size)
        if rank == size-1:
            up_limit = mat_size

        M = np.zeros((mat_size, mat_size))
        M_1d_size = int(mat_size * (mat_size + 1)/2)
        #eig = np.zeros(mat_size)
        Mdiag = np.zeros(mat_size)
        Mdiag_local = np.zeros(mat_size)
        M_local = np.zeros((mat_size, mat_size))
        for i in range(lo_limit, up_limit):
            Mdiag_local[i] = circuit_d(params, list1[i], wires, null_state, ash_excitation)
        comm.Reduce(Mdiag_local, Mdiag, MPI.SUM, 0)

        def idx(n):
            id_i = np.ceil((-1 + np.sqrt(1 + 4 * 2 * n))/2)
            id_j = n - (id_i-1) * (id_i)/2
            return int(id_i-1), int(id_j-1)

        for ii in range(rank, M_1d_size, size):
            c = ii + 1

            idi, idj = idx(c)

            if idi==idj:
                Mtmp = Mdiag[idi]
            else:
                #print(idi, idj)
                Mtmp = circuit_od(params, list1[idi], list1[idj], wires, null_state, ash_excitation) - Mdiag[idi]/2.0 - Mdiag[idj]/2.0

            M_local[idi, idj] = Mtmp
            M_local[idj, idi] = Mtmp

        comm.Reduce(M_local, M, MPI.SUM, 0)
        comm.Bcast(M, root=0)
    
        eig, evec=np.linalg.eig(M)
      
        return np.sort(eig), evec

    print(f"Computing for bond length: {d / r:.2f} Å")
    params, ash_excitation = gs_exact(symbols, geometry, active_electrons, active_orbitals, charge, adapt_it=10) #1 is used for params
    eig, evec = qsc_eom(symbols, geometry, active_electrons, active_orbitals, charge, params, ash_excitation, shots = 200)
#np.save('eigenvalues.npy', eig)
#np.save('eigenvectors.npy', evec)
    excited_state_energies.append(eig)
    print("qscEOM200", excited_state_energies)

Computing for bond length: 0.60 Å
HF state is -55.50579854955699


/Users/prince.kwao/miniconda3/envs/prince/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:353: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


gr_energies: -55.519648902656144
qscEOM200 [tensor([-55.57252776, -55.12705335, -55.08449233, -55.01211865,
        -54.95555063, -54.80852486, -54.7597398 , -54.69370481,
        -54.59212606, -54.46352728, -54.41363479, -54.35610988,
        -54.31187671, -54.25743977, -54.25198115, -54.14623727,
        -54.09529061, -54.06355911, -53.98870705, -53.96043496,
        -53.91799116, -53.87506835, -53.81534308, -53.76488703,
        -53.6795627 , -53.5805692 , -53.54474027], requires_grad=True)]
Computing for bond length: 0.70 Å
HF state is -55.7762385068465
gr_energies: -55.79038457104941
qscEOM200 [tensor([-55.57252776, -55.12705335, -55.08449233, -55.01211865,
        -54.95555063, -54.80852486, -54.7597398 , -54.69370481,
        -54.59212606, -54.46352728, -54.41363479, -54.35610988,
        -54.31187671, -54.25743977, -54.25198115, -54.14623727,
        -54.09529061, -54.06355911, -53.98870705, -53.96043496,
        -53.91799116, -53.87506835, -53.81534308, -53.76488703,
        -

In [5]:
import os
from time import time
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import re
import scipy
from mpi4py import MPI
#from qiskit_ibm_runtime.fake_provider import FakeManilaV2, FakeMelbourneV2, FakeMumbaiV2, FakeBrisbane, FakeTorino
#from qiskit_ibm_runtime.options import ZneOptions
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
#from qiskit_aer import AerSimulator
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity

import scipy



comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

def inite(elec,orb):
    config=[]
    list1=[]
    list1.append([e for e in range(elec)])
    #singles
    for x in range(elec):
        count=orb-elec
        while (count<orb):
            for e in range(elec):
                if x==e:
                    if x%2==0:
                        config.append(count)
                        count=count+2
                    else:
                        config.append(count+1)
                        count=count+2
                else:
                    config.append(e)

            list1.append(config)
            config=[]
    #doubles
    for x in range(elec):
        for y in range(x+1,elec):
            count1=orb-elec
            count2=orb-elec
            for count1 in range(elec, orb, 2):
                for count2 in range(elec, orb, 2):
                    cont=0
                    if count1==count2:
                        if (x%2)!=(y%2):
                            cont=1
                    else:
                        cont=1
                    if (x%2)==(y%2) and count2<count1:
                        cont=0
                    if cont==1:
                        for e in range(elec):
                            if x==e:
                                if x%2==0:
                                    config.append(count1)
                                else:
                                    config.append(count1+1)
                            elif y==e:
                                if y%2==0:
                                    config.append(count2)
                                else:
                                    config.append(count2+1)
                            else:
                                config.append(e)
                        list1.append(config)
                        config=[]
    return list1
r = 1.88973  # 1 Angstrom in Bohr
distances = np.linspace(0.6*r, 2.46*r, 19)  # from 1 to bohr

# Setup other constants
symbols  = ['N', 'H', 'H', 'H']
#gr_energies = []
excited_state_energies = []

    # Define the molecule
for d in distances:
    #geometry = np.array([[0.0, 0.0,  d],
                        #[0.0, 0.0, -0.5488*r]
                        #], requires_grad = False)
    #active_electrons = 6
    #active_orbitals = 6
    #charge = 0
    
    geometry = np.array([[0.0, 0.0,  0.0],
                        [d, 0.0, 0.0],
                        [-0.506*r, 0.876*r, 0.0],
                        [-0.506*r, -0.876*r, 0.0]
                        ], requires_grad = False)
    active_electrons = 4
    active_orbitals = 4
    charge = 0
  

    def gs_exact(symbols, coordinates, active_electrons, active_orbitals, charge, adapt_it):
        gr_energies = []
        ash_excitation = []
        H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
        
        hf_state = qml.qchem.hf_state(active_electrons, qubits)
        #Calculation of HF state
        dev = qml.device("lightning.qubit", wires=qubits)
        @qml.qnode(dev)
        def circuit(hf_state, active_electrons, qubits, H):
            #print('Updated hf_state is', hf_state)  
            qml.BasisState(hf_state, wires=range(qubits))
            return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
        
        # Commutator calculation for HF state
        @qml.qnode(dev)
        def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
            qml.BasisState(k, wires=range(qubits))
            res = qml.commutator(H, w)   #Calculating the commutator
            return qml.expval(res)
        
        # Commutator calculation for other states except HF state
        @qml.qnode(dev)
        def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
            qml.StatePrep(k, wires=range(qubits))
            res = qml.commutator(H, w) #Calculating the commutator
            return qml.expval(res)

        #Energy calculation 
        @qml.qnode(dev)
        def ash(params, ash_excitation, hf_state, H):
            [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
            for i, excitation in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
        
        # Calculation of New state, same as the above function but with the state return
        dev1 = qml.device("lightning.qubit", wires=qubits)
        @qml.qnode(dev1)
        def new_state(hf_state, ash_excitation, params):
            [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
            for i, excitations in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.state()
        
        def cost(params):
            energy = ash(params, ash_excitation, hf_state, H)
            return energy

        print('HF state is', circuit(hf_state, active_electrons, qubits, H))
        singles, doubles = qml.qchem.excitations(active_electrons, qubits)
        op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
        op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
        operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
        #print('Total excitations are', len(operator_pool))
        states = [hf_state]
        params = np.zeros(len(ash_excitation), requires_grad=True) 

        null_state = np.zeros(qubits,int)
        #print('Null state is', null_state)

        for j in range(adapt_it):
            #print('The adapt iteration now is', j)  #Adapt iteration
            max_value = float('-inf')
            max_operator = None
            k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
        
            for i in operator_pool:
                #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
                w = qml.fermi.jordan_wigner(i)  #JW transformation
                if np.array_equal(k, hf_state): # If the current state is the HF state
                    current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
                else:
                    current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

                if current_value > max_value:
                    max_value = current_value
                    max_operator = i

            indices_str = re.findall(r'\d+', str(max_operator))
            excitations = [int(index) for index in indices_str]
            #print('Highest gradient excitation is', excitations)
            ash_excitation.append(excitations) #Appending the excitations to the ash_excitation

            params = np.append(params, 0.0)  #Parameters initialization

            #Energy calculation
            result = minimize(cost, params, method='powell', tol = 1e-12, options = {'disp': False, 'maxiter': 1e8})

            params= (result.x)
            gr_energies.append(result.fun)


            ostate = new_state(hf_state, ash_excitation, params)
           
            states.append(ostate)
        print("gr_energies:", gr_energies[-1])
        return params, ash_excitation



    def qsc_eom(symbols, coordinates, active_electrons, active_orbitals, charge, params, ash_excitation, shots=0):
        # Build the electronic Hamiltonian
        H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method='pyscf', active_electrons=active_electrons, active_orbitals=active_orbitals, charge=charge)
        hf_state = qml.qchem.hf_state(active_electrons, qubits)
        
        wires=range(qubits)

        null_state = np.zeros(qubits,int)
        list1 = inite(active_electrons,qubits)
        if shots==0:
            dev = qml.device("lightning.qubit", wires=qubits)
           
        else:
            dev = qml.device("lightning.qubit", wires=qubits,shots=shots)
            
        #circuit for diagonal part
        @qml.qnode(dev)
        def circuit_d(params, occ,wires, hf_state, ash_excitation):
            for w in occ:
                qml.X(wires=w)
            for i, excitations in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.expval(H)
        
        #circuit for off-diagonal part
        @qml.qnode(dev)
        def circuit_od(params, occ1, occ2, wires, hf_state, ash_excitation):
            for w in occ1:
                qml.X(wires=w)
            first=-1
            for v in occ2:
                if v not in occ1:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            for v in occ1:
                if v not in occ2:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            for i, excitations in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.expval(H)

        #final M matrix
        mat_size = len(list1)
        
        block_size = mat_size/size
        lo_limit = int(rank * block_size)
        up_limit = int((rank+1) * block_size)
        if rank == size-1:
            up_limit = mat_size

        M = np.zeros((mat_size, mat_size))
        M_1d_size = int(mat_size * (mat_size + 1)/2)
        #eig = np.zeros(mat_size)
        Mdiag = np.zeros(mat_size)
        Mdiag_local = np.zeros(mat_size)
        M_local = np.zeros((mat_size, mat_size))
        for i in range(lo_limit, up_limit):
            Mdiag_local[i] = circuit_d(params, list1[i], wires, null_state, ash_excitation)
        comm.Reduce(Mdiag_local, Mdiag, MPI.SUM, 0)

        def idx(n):
            id_i = np.ceil((-1 + np.sqrt(1 + 4 * 2 * n))/2)
            id_j = n - (id_i-1) * (id_i)/2
            return int(id_i-1), int(id_j-1)

        for ii in range(rank, M_1d_size, size):
            c = ii + 1

            idi, idj = idx(c)

            if idi==idj:
                Mtmp = Mdiag[idi]
            else:
                #print(idi, idj)
                Mtmp = circuit_od(params, list1[idi], list1[idj], wires, null_state, ash_excitation) - Mdiag[idi]/2.0 - Mdiag[idj]/2.0

            M_local[idi, idj] = Mtmp
            M_local[idj, idi] = Mtmp

        comm.Reduce(M_local, M, MPI.SUM, 0)
        comm.Bcast(M, root=0)
    
        eig, evec=np.linalg.eig(M)
      
        return np.sort(eig), evec

    print(f"Computing for bond length: {d / r:.2f} Å")
    params, ash_excitation = gs_exact(symbols, geometry, active_electrons, active_orbitals, charge, adapt_it=10) #1 is used for params
    eig, evec = qsc_eom(symbols, geometry, active_electrons, active_orbitals, charge, params, ash_excitation, shots = 100)
#np.save('eigenvalues.npy', eig)
#np.save('eigenvectors.npy', evec)
    excited_state_energies.append(eig)
    print("qscEOM100", excited_state_energies)

Computing for bond length: 0.60 Å
HF state is -55.50579854955699


/Users/prince.kwao/miniconda3/envs/prince/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:353: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


gr_energies: -55.519648902656144
qscEOM100 [tensor([-55.56653859, -55.12693085, -55.10007239, -55.03452236,
        -55.01762865, -54.94422839, -54.77031243, -54.65748032,
        -54.55358372, -54.51134086, -54.41812384, -54.36070101,
        -54.29013218, -54.21959994, -54.19982411, -54.14436501,
        -54.06480073, -54.03298958, -53.96627342, -53.95578053,
        -53.89452697, -53.87404761, -53.84497876, -53.80225923,
        -53.75624269, -53.54542487, -53.47759485], requires_grad=True)]
Computing for bond length: 0.70 Å
HF state is -55.7762385068465
gr_energies: -55.79038457104941
qscEOM100 [tensor([-55.56653859, -55.12693085, -55.10007239, -55.03452236,
        -55.01762865, -54.94422839, -54.77031243, -54.65748032,
        -54.55358372, -54.51134086, -54.41812384, -54.36070101,
        -54.29013218, -54.21959994, -54.19982411, -54.14436501,
        -54.06480073, -54.03298958, -53.96627342, -53.95578053,
        -53.89452697, -53.87404761, -53.84497876, -53.80225923,
        -